# metadata4Ing

[Metadata4Ing](https://nfdi4ing.pages.rwth-aachen.de/metadata4ing/metadata4ing/) is an ontology developed within the NFDI Consortium [NFDI4Ing](https://nfdi4ing.de/) "with the aim of providing a thorough framework for the semantic description of research data, with a particular focus on engineering sciences and neighbouring disciplines."

In [1]:
import h5rdmtoolbox as h5tbx
_ = h5tbx.set_config(auto_create_h5tbx_version=False)

## Simple example using m4i:NumericalVariable

The most basic example of using M4I with HDF5 is to semantify an HDF5 dataset with [m4i:NumericalVariable](http://w3id.org/nfdi4ing/metadata4ing#NumericalVariable)

In [2]:
from ontolutils.namespacelib import M4I, QUDT_UNIT

In [3]:
with h5tbx.File() as h5:
    ds = h5.create_dataset("velocity", data=4.5, attrs={"units": "m/s"})
    ds.rdf.type = M4I.NumericalVariable
    ds.rdf.data_predicate = M4I.hasNumericalValue
    ds.rdf["units"].predicate = M4I.hasUnit
    ds.rdf["units"].object = QUDT_UNIT.M_PER_SEC

    h5.dump(collapsed=False)

    print(h5.serialize("ttl", structural=False, file_uri="https://example.org#"))

@prefix m4i: <http://w3id.org/nfdi4ing/metadata4ing#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://example.org#tmp0.hdf/velocity> a m4i:NumericalVariable ;
    m4i:hasNumericalValue "4.5"^^xsd:float ;
    m4i:hasUnit <http://qudt.org/vocab/unit/M-PER-SEC> .




## Reading in the Metadata4Ing-Example

The *h5RDMtoolbox* allows translating JSON-LD data into the HDF5 structure and wise versa (see [here](../userguide/wrapper/FAIRAttributes.ipynb)). This way, engineering process metadata can be added easily to the HDF5 file. 

Consider the following example file taken from the *m4i* ontology ["first steps guide"](https://git.rwth-aachen.de/nfdi4ing/metadata4ing/metadata4ing/-/blob/1.3.1/training/first-steps-guide.md), which describes an experimental investigation. You can find the full content of the JSON-LD file [here](min_m4i_ex.jsonld), the following shows an excerpt:

In [4]:
import h5rdmtoolbox as h5tbx
_ = h5tbx.set_config(auto_create_h5tbx_version=False)

With a single call (`h5tbx.jsonld.to_hdf`) it is possible to read in the JSON-LD data and parse it into the HDF5 structure:

In [5]:
with h5tbx.File() as h5:
    h5.create_group('metadata')
    h5tbx.jsonld.to_hdf(h5.metadata, source='min_m4i_ex.jsonld')

ValueError: Can't split 'person'

In [ ]:
print(h5tbx.serialize(h5.hdf_filename, format="ttl", structural=False, file_uri="https://local-domain.org/"))

In [ ]:
print(
    h5tbx.serialize(
        h5.hdf_filename,
        format="ttl",
        structural=False,
        file_uri="https://local-domain.org/",
        context={"@import": "https://w3id.org/nfdi4ing/metadata4ing/m4i_context.jsonld"})
)